<a href="https://colab.research.google.com/github/Hsabonchi/-Firefox-Test-automation/blob/master/py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1337]:
# pip install yfinance
# pip install requests_cache


In [1338]:

import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time 
import yfinance as yf
import requests_cache
%matplotlib inline


In [1339]:
# ___variables___
ticker = 'BA'
# start and end date of historical data
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2020, 12, 29)

# you need this line for real live data
# end = datetime.datetime.now().date().isoformat()         # today


# To avoid yahoo finnaince blocking
expire_after = datetime.timedelta(days=3)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)


In [1340]:
# yahoo gives only daily historical data server
connected = False
while not connected:
    try:
        ticker_df = web.get_data_yahoo(ticker, start, end,interval ="d",session=session)
        connected = True
        print('connected to yahoo')
    except Exception as e:
        print("type error: " + str(e))
        time.sleep( 5 )
        pass   

# use numerical integer index instead of date    
ticker_df = ticker_df.reset_index()
# print(ticker_df.head(5))

connected to yahoo


### Compute RSI
#### RSI Method

In [1341]:
df = ticker_df
def RSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)
    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi
df['RSI'] = RSI(df['Close'], 14)

#### Calculating ROC 

In [1342]:
def ROC(df, n):  
    M = df.diff(n)  
    N = df.shift(n)  
    ROC = pd.Series(((M / N) * 100), name = 'ROC_' + str(n))   
    return ROC
    
df['ROC'] = ROC(df['Close'], 9)

In [1343]:
def isRSI(indexOfdf,oversold=30,oversoldThreshold=33,overbought=70,overboughtThreshold=73):
          
          # Time to buy , check if the RSI within the range of 30 - 33
          if float(df.iloc[indexOfdf, :]['RSI']) >= float(oversold) and (df.iloc[indexOfdf, :]['RSI']) <= float(oversoldThreshold):
              current_RSI1 = df.iloc[indexOfdf, :]['RSI']
              prev_RSI1 = df.iloc[indexOfdf-1, :]['RSI']  
              if (prev_RSI1 < current_RSI1 ): 
                      return True       
              # time to sell
              elif float(df.iloc[indexOfdf, :]['RSI']) >= float(overbought) and (df.iloc[indexOfdf, :]['RSI']) <= float(overboughtThreshold):
                      current_RSI2 = df.iloc[indexOfdf, :]['RSI']
                      prev_RSI2 = df.iloc[indexOfdf-1, :]['RSI']
                      if  prev_RSI2 > current_RSI2:
                              return False
                                    

In [1344]:
def isROC(index):
    # ROC less than zero to buy  means true when we call the isROC(index)

    return df.iloc[index,:]['ROC'] < 0


In [1345]:
### startDate is the starting date to buy.
## endDate  is the ending date to buy
## buyingPower is a budget that customer wants to invest
### dayThreshold. "Number of stockes to buy per day"
### weekThreshold "Number of stockes to buy per week"
### spendingThreshold per month 
### isrealTime=False False means going over historical data.

def buyStock (startDate,endDate,buyingPower,spendingThreshold,dayThreshold=1,weekThreshold=5,isrealTime=False,oversold=30,oversoldThreshold=33,overbought=70,overboughtThreshold=73):
  print(f"startDate{startDate}: endDate {endDate} ")
  totalNumStock=0
  # check the funds first
  if buyingPower > 0:
      if isrealTime == True:
          print("Working on real time data")
        # Working on offline data.
      else:   # loop over the dataframe  to get the start date.
            print('looping over the dataframe  to get the start date.')
            for index, row in df.iterrows():
                  date=df.iloc[index, :]['Date'].date().strftime('%Y-%m-%d') 
                  dayThresholdCount=0
                  # print(f"outise if {index} :  {date}")
                  if startDate >= date:
                            # print(f"insise loop {index} :{date}")
                                # Check the RSI & ROC (later we will add more indicator in this statment)
                            if isRSI(index) == True and isROC(index)== True:
                                 
                                        # check the buyingPower is greater than or equal price
                                  if buyingPower >=row ['Close']:
                                          # Check if we reached to the limit of the number of stock in a specific day(dayThreshold)
                                          if dayThresholdCount < dayThreshold:
                                                # increment the "dayThresholdCount" by dayThreshold
                                                dayThresholdCount=dayThresholdCount+dayThreshold
                                                totalNumStock=totalNumStock+dayThreshold
                                                # Decrement buyingPower by closing price
                                                buyingPower=buyingPower-row ['Close']
                                                print (f"bought a stck {row ['Close']} on: {date} mybuget is - {buyingPower} ")
                                                

                                                
                          
                                          else:      
                                              print ("You have reached your daily threshold ") 
                                              return  buyingPower,totalNumStock,dayThresholdCount        
                            else:
                              continue
                                  # print (f"RSI and ROC do not allow us to buy {date}")
                       
  else:
      print("Buying Power is not sufficient")





In [1346]:
buyStock('2020-03-04','2020-03-30',600,100)


startDate2020-03-04: endDate 2020-03-30 
looping over the dataframe  to get the start date.
bought a stck 122.08000183105469 on: 2014-12-15 mybuget is - 477.9199981689453 
bought a stck 141.49000549316406 on: 2015-05-07 mybuget is - 336.42999267578125 
bought a stck 129.33999633789062 on: 2015-08-26 mybuget is - 207.08999633789062 
bought a stck 129.1999969482422 on: 2016-01-14 mybuget is - 77.88999938964844 


In [1347]:
 isROC(1804)== True

True

In [1348]:
# 1) main functio (interval h,d,w)
  # get data and create df
  #calucalte RSI and ROC
df.iloc[1804, :]['ROC']

-15.80825617196421

In [1349]:
df.iloc[1810, :]['Date'].date().strftime('%Y-%m-%d')=='2020-03-04'

False

In [1350]:
#df.loc[[20],['Date']].strftime('%Y-%m-%d')=='2020-03-07'
df.iloc[20, :]['Date'].date().strftime('%Y-%m-%d')=='2020-03-07'

False

In [1351]:
# def buyStock():
#     return 'abc', 10
# result=test()
# result[1]

In [1352]:
# def buyStock (startDate,endDate,buyingPower,spendingThreshold,dayThreshold=1,weekThreshold=5,isrealTime=False,oversold=30,oversoldThreshold=33,overbought=70,overboughtThreshold=73):
#   print(f"startDate{startDate}: endDate {endDate} ")
#   totalNumStock=0
#   # check the funds first
#   if buyingPower > 0:
#     if isrealTime == True:
#         print("Working on real time data")
#       # Working on offline data.
#       else:   # loop over the dataframe  to get the start date.
#               for index, row in df.iterrows():
#                 #daily counter
#                 dayThresholdCount=0
#                 if row['Date'].date().strftime('%Y-%m-%d')==startDate and row['Date'].date().strftime('%Y-%m-%d')<= endDate:
#                               date=row['Date'].date() 
#                               # debugging purpose
#                               # print(date)
#                               # Check the RSI & ROC (later we will add more indicator in this statment)
#                               print(index)
#                               if isRSI(index) == True and isROC(index)== True:
#                                 print ("isRSI(index) == True and isROC(index)== True]")
#                                 # check the buyingPower is greater than or equal price
#                                 if buyingPower >=row ['Close']:
#                                   print ("buyingPower >=row ['Close']")
#                                   # Check if we reached to the limit of the number of stock in a specific day(dayThreshold)
#                                   if dayThresholdCount < dayThreshold:
#                                         print ("dayThresholdCount < dayThreshold")
#                                         # increment the "dayThresholdCount" by dayThreshold
#                                         dayThresholdCount=dayThresholdCount+dayThreshold
#                                         totalNumStock=totalNumStock+dayThreshold
#                                         # Decrement buyingPower by closing price
#                                         buyingPower=buyingPower-row ['Close']
#                                         print ("buy ")
#                                         # return  buyingPower,totalNumStock,dayThresholdCount
                  
#                                   else:      
#                                       print ("You have reached your daily threshold ")
#                                 else:
#                                      print ("Buying Power is not sufficient")
#                                      return 
#                                      break

      
#   else:
#       print(f"RSI and ROC do not allow us to buy ")